## This notebook will work on converting text to SQL queries that can be run on Yelp dataset using an SQL agent

### Install Dependencies

In [1]:
%pip --quiet install langchain psycopg2 jq faiss-gpu transformers openai tiktoken python-environ

Note: you may need to restart the kernel to use updated packages.


### Import libraries

In [2]:
import os
import json
import environ
from typing import List
from langchain_community.llms import OpenAI
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

from config import DATABASE_NAME, USERNAME, PASSWORD, TABLES

### Database connection string

In [3]:
HOST = "localhost"
PORT = "5432"
pg_uri = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"

### Load the LLM

In [4]:
env = environ.Env()
environ.Env.read_env()
API_KEY = env('OPENAI_API_KEY')

llm = OpenAI(openai_api_key=API_KEY, temperature=0.0)

/home/prasad/.local/lib/python3.8/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_7796/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(
/home/prasad/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


### Initialize the embeddings class

In [5]:
embeddings = OpenAIEmbeddings()

/home/prasad/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Prepare schema documents

In [6]:
tables_document = JSONLoader(file_path='./schemas/tables.jsonl', jq_schema='.', text_content=False, json_lines=True).load()
columns_document = JSONLoader(file_path='./schemas/columns.jsonl', jq_schema='.', text_content=False, json_lines=True).load()

### Helper method for retrieving matched tables from vector store

In [7]:
def get_matched_tables(query: str) -> List[Document]:
    db = FAISS.from_documents(documents=tables_document, embedding=embeddings)
    retriever = db.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 1})
    matched_documents = retriever.get_relevant_documents(query=query)

    return matched_documents

### Helper method for retrieving matched columns from vector store

In [8]:
def get_matched_columns(query: str) -> List[Document]:
    db = FAISS.from_documents(documents=columns_document, embedding=embeddings)
    search_kwargs = {
        'k': 20
    }

    retriever = db.as_retriever(search_type='similarity', search_kwargs=search_kwargs)
    matched_columns = retriever.get_relevant_documents(query=query)

    return matched_columns

### Construct the system prompt

In [9]:
system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct Postgres query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer."""

### Build the agent

In [12]:
def find_tables_to_include(query: str) -> tuple:
    table_info = {}
    tables = set()
    matched_columns = get_matched_columns(query)
    for i in matched_columns:
        # print(i)
        content = json.loads(i.page_content)
        tables.add(content['table_name'])
        if content['table_name'] == 'businesses':
            if 'businesses' in table_info.keys():
                schema = f"""{table_info['businesses']}
                            {content['column_name']}: {content['description']} {content['usage']}
                            """
                table_info['businesses'] = schema
            else:
                table_info['businesses'] = f"""{TABLES[0]}
                                        {content['column_name']}: {content['description']} {content['usage']}
                                        """
        
        elif content['table_name'] == 'users':
            if 'users' in table_info.keys():
                schema = f"""{table_info['users']}
                            {content['column_name']}: {content['description']} {content['usage']}
                            """
                table_info['users'] = schema
            else:
                table_info['users'] = f"""{TABLES[1]}
                                        {content['column_name']}: {content['description']} {content['usage']}
                                        """
                
        elif content['table_name'] == 'reviews':
            if 'reviews' in table_info.keys():
                schema = f"""{table_info['reviews']}
                            {content['column_name']}: {content['description']} {content['usage']}
                            """
                table_info['reviews'] = schema
            else:
                table_info['reviews'] = f"""{TABLES[2]}
                                        {content['column_name']}: {content['description']} {content['usage']}
                                        """
                
        elif content['table_name'] == 'checkins':
            if 'checkins' in table_info.keys():
                schema = f"""{table_info['checkins']}
                            {content['column_name']}: {content['description']} {content['usage']}
                            """
                table_info['checkins'] = schema
            else:
                table_info['checkins'] = f"""{TABLES[3]}
                                        {content['column_name']}: {content['description']} {content['usage']}
                                        """
                
        elif content['table_name'] == 'tips':
            if 'tips' in table_info.keys():
                schema = f"""{table_info['tips']}
                            {content['column_name']}: {content['description']} {content['usage']}
                            """
                table_info['tips'] = schema
            else:
                table_info['tips'] = f"""{TABLES[4]}
                                        {content['column_name']}: {content['description']} {content['usage']}
                                        """

    # print(table_info)
    return list(tables), table_info

def execute_query(query: str):
    tables_to_include, table_info = find_tables_to_include(query)
    print(tables_to_include)
    
    db = SQLDatabase.from_uri(pg_uri, include_tables=tables_to_include, custom_table_info=table_info)
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)

    agent_executor = create_sql_agent(llm=llm, 
                                  toolkit=toolkit,  
                                  prefix=system_prefix,
                                  return_intermediate_steps=True, 
                                  verbose=True)

    return agent_executor.invoke(query)

In [14]:
execute_query("Find me the user who has posted the coolest review?")

['reviews', 'tips', 'businesses', 'users']


> Entering new AgentExecutor chain...


/home/prasad/.local/lib/python3.8/site-packages/langchain_community/agent_toolkits/sql/base.py:138: UserWarning: Received additional kwargs {'return_intermediate_steps': True} which are no longer supported.
  warnings.warn(


 We need to find the table that contains user information and review information, and then join them together to find the coolest review.
Action: sql_db_list_tables
Action Input: ""businesses, reviews, tips, usersWe need to find the table that contains user information and review information, and then join them together to find the coolest review.
Action: sql_db_schema
Action Input: users, reviews
    CREATE TABLE reviews (
        review_id VARCHAR(22) PRIMARY KEY NOT NULL, 
        user_id VARCHAR(22) NOT NULL REFERENCES users (user_id), 
        business_id VARCHAR(22) NOT NULL REFERENCES businesses (business_id), 
        stars INT NOT NULL, 
        date DATE NOT NULL, 
        text TEXT NOT NULL, 
        useful INT, 
        funny INT, 
        cool INT
    )
    
                                        cool: Number of users who found the review cool or the number of cool votes received by this review. Displaying the number of users who found the review cool.
                   

{'input': 'Find me the user who has posted the coolest review?',
 'output': "The user who has posted the coolest review is 'Ormond'."}

In [90]:
execute_query("Which user from Boston checked in into a spa?")

/home/prasad/.local/lib/python3.8/site-packages/langchain_community/agent_toolkits/sql/base.py:138: UserWarning: Received additional kwargs {'return_intermediate_steps': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...
 I need to query the user and check-in tables to find the necessary information.
Action: sql_db_list_tables
Action Input: ""businesses, checkins, reviews, tips, usersI need to use the user and check-in tables to get the relevant data.
Action: sql_db_schema
Action Input: user, checkinError: table_names {'checkin', 'user'} not found in databaseI need to double check if the table names are correct using sql_db_query_checker.
Action: sql_db_query_checker
Action Input: SELECT * FROM checkins
SELECT * FROM checkins
--No common mistakes in this query I need to add the necessary columns and conditions to the query.
Action: sql_db_query
Action Input: SELECT * FROM checkins WHERE city = 'Boston' AND category = 'spa'Error: (psycopg2.errors.UndefinedColumn) column "city" does not exist
LINE 1: SELECT * FROM checkins WHERE city = 'Boston' AND category = ...
                                     ^

[SQL: SELECT * FROM checkins WHERE city = 'Boston' AND category

{'input': 'Which user from Boston checked in into a spa?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [91]:
execute_query("Find me the business with the funniest review in San Francisco and get me the user who posted the review as well")

/home/prasad/.local/lib/python3.8/site-packages/langchain_community/agent_toolkits/sql/base.py:138: UserWarning: Received additional kwargs {'return_intermediate_steps': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...
 We should start by looking at the reviews and finding the ones with the highest ratings, then filter by location and business type.
Action: sql_db_query
Action Input: SELECT business_name, user_name, review_rating FROM business_reviews WHERE review_rating = 5 AND city = 'San Francisco' AND business_type = 'funny'Error: (psycopg2.errors.UndefinedTable) relation "business_reviews" does not exist
LINE 1: ...LECT business_name, user_name, review_rating FROM business_r...
                                                             ^

[SQL: SELECT business_name, user_name, review_rating FROM business_reviews WHERE review_rating = 5 AND city = 'San Francisco' AND business_type = 'funny']
(Background on this error at: https://sqlalche.me/e/20/f405)We should check if the table name is correct and if the columns exist in the table.
Action: sql_db_schema
Action Input: business_reviewsError: table_names {'business_reviews'} not found in databaseWe should c

{'input': 'Find me the business with the funniest review in San Francisco and get me the user who posted the review as well',
 'output': "SELECT business_name, user_name, review_rating FROM business_reviews WHERE review_rating = 5 AND city = 'San Francisco' AND business_type = 'funny'"}

In [11]:
execute_query("find me the business with the highest stars")

/home/prasad/.local/lib/python3.8/site-packages/langchain_community/agent_toolkits/sql/base.py:138: UserWarning: Received additional kwargs {'return_intermediate_steps': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...
 I need to find the table that contains business information and the table that contains review information. Then I can join the two tables and sort by the stars column.
Action: sql_db_list_tables
Action Input: ""businesses, reviews, tips, usersI need to use the businesses and reviews tables.
Action: sql_db_schema
Action Input: businesses, reviews
    CREATE TABLE businesses (
        business_id VARCHAR(22) PRIMARY KEY NOT NULL, 
        name TEXT NOT NULL, 
        address TEXT NOT NULL, 
        city TEXT NOT NULL, 
        state TEXT NOT NULL, 
        postal_code TEXT NOT NULL, 
        latitude DOUBLE PRECISION, 
        longitude DOUBLE PRECISION, 
        stars FLOAT, 
        review_count INT, 
        is_open BOOLEAN NOT NULL, 
        categories TEXT[], 
        monday_hours TEXT, 
        tuesday_hours TEXT, 
        wednesday_hours TEXT, 
        thursday_hours TEXT, 
        friday_hours TEXT, 
        satday_hours TEXT, 
        su